<a href="https://colab.research.google.com/github/Carrambert/Shakespear-mining/blob/main/Trab_BD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar as dependências

#Instalar Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Realizar o download do Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

#Descompartar o arquivo baixado
!tar xf spark-3.4.3-bin-hadoop3.tgz

#Instalando a findspark
!pip install -q findspark

In [ ]:
! pip install nltk pandas
! python -m nltk.downloader wordnet omw-1.4

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#Configurar as variáveis de ambiente

#Importando a biblioteca os
import os

#Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

#Importando a findspark
import findspark

#Iniciando o findspark
findspark.init('spark-3.4.3-bin-hadoop3')

In [ ]:
from pyspark.sql.functions import regexp_replace, col, lower, split, explode, count, translate, udf, monotonically_increasing_id
from pyspark.ml.feature import StopWordsRemover, Tokenizer
import string
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import ArrayType, StringType, IntegerType
from pyspark.sql import SparkSession
import time

In [ ]:
# iniciar uma sessão local
from pyspark.sql import SparkSession
import time
sc = SparkSession.builder.master('local[*]').config('spark.ui.port', '4050').getOrCreate()

In [ ]:
import nltk

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
arquivo = "Shakespeare_alllines.txt"


with open(arquivo, 'r') as file:

    lines = file.readlines()


shows = ",".join(lines).replace('"', "").replace("'", "").replace('“', "").replace('”', "").replace("--", "").replace("SCENE", "").replace(".", "").replace("ACT II", "").replace("ACT III", "").replace("ACT IV", "").split("ACT I")
print(shows[1:10])
#data = sc.read.text(arquivo)

['\n, I London The palace\n,Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others\n,So shaken as we are, so wan with care,\n,Find we a time for frighted peace to pant,\n,And breathe short-winded accents of new broils\n,To be commenced in strands afar remote\n,No more the thirsty entrance of this soil\n,Shall daub her lips with her own childrens blood,\n,Nor more shall trenching war channel her fields,\n,Nor bruise her flowerets with the armed hoofs\n,Of hostile paces: those opposed eyes,\n,Which, like the meteors of a troubled heaven,\n,All of one nature, of one substance bred,\n,Did lately meet in the intestine shock\n,And furious close of civil butchery\n,Shall now, in mutual well-beseeming ranks,\n,March all one way and be no more opposed\n,Against acquaintance, kindred and allies:\n,The edge of war, like an ill-sheathed knife,\n,No more shall cut his master Therefore, friends,\n,As far as to the sepulchre of Christ,\n,Whose soldier now, un

In [ ]:
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
play1 = nltk.word_tokenize(shows[1])

tokenized_plays = [nltk.word_tokenize(play) for play in shows]

stop_words = set(stopwords.words('english'))

def remove_punctuation_and_stopwords(tokens):
    return [word for word in tokens if word not in string.punctuation and word.lower() not in stop_words]


cleaned_plays = [remove_punctuation_and_stopwords(tokens) for tokens in tokenized_plays]
print(cleaned_plays[1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['London', 'palace', 'Enter', 'KING', 'HENRY', 'LORD', 'JOHN', 'LANCASTER', 'EARL', 'WESTMORELAND', 'SIR', 'WALTER', 'BLUNT', 'others', 'shaken', 'wan', 'care', 'Find', 'time', 'frighted', 'peace', 'pant', 'breathe', 'short-winded', 'accents', 'new', 'broils', 'commenced', 'strands', 'afar', 'remote', 'thirsty', 'entrance', 'soil', 'Shall', 'daub', 'lips', 'childrens', 'blood', 'shall', 'trenching', 'war', 'channel', 'fields', 'bruise', 'flowerets', 'armed', 'hoofs', 'hostile', 'paces', 'opposed', 'eyes', 'like', 'meteors', 'troubled', 'heaven', 'one', 'nature', 'one', 'substance', 'bred', 'lately', 'meet', 'intestine', 'shock', 'furious', 'close', 'civil', 'butchery', 'Shall', 'mutual', 'well-beseeming', 'ranks', 'March', 'one', 'way', 'opposed', 'acquaintance', 'kindred', 'allies', 'edge', 'war', 'like', 'ill-sheathed', 'knife', 'shall', 'cut', 'master', 'Therefore', 'friends', 'far', 'sepulchre', 'Christ', 'Whose', 'soldier', 'whose', 'blessed', 'cross', 'impressed', 'engaged', 'fig

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
nltk.download('wordnet')
nltk.download('omw-1.4')


lemmatizer = WordNetLemmatizer()


def remove_punctuation_stopwords_and_lemmatize(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens if word not in string.punctuation and word.lower() not in stop_words]


cleaned_plays = [remove_punctuation_stopwords_and_lemmatize(tokens) for tokens in tokenized_plays]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN




def remove_punctuation_stopwords_and_classify(tokens, idx):

    filtered_tokens = [word for word in tokens if word not in string.punctuation and word.lower() not in stop_words]


    pos_tagged_tokens = pos_tag(filtered_tokens)


    classified_tokens = [(word, get_wordnet_pos(tag), idx) for word, tag in pos_tagged_tokens]

    return classified_tokens


classified_plays_with_index = [remove_punctuation_stopwords_and_classify(tokens, idx) for idx, tokens in enumerate(tokenized_plays)]

print(classified_plays_with_index[1])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('London', 'n', 1), ('palace', 'n', 1), ('Enter', 'n', 1), ('KING', 'n', 1), ('HENRY', 'n', 1), ('LORD', 'n', 1), ('JOHN', 'n', 1), ('LANCASTER', 'n', 1), ('EARL', 'n', 1), ('WESTMORELAND', 'n', 1), ('SIR', 'n', 1), ('WALTER', 'n', 1), ('BLUNT', 'n', 1), ('others', 'n', 1), ('shaken', 'v', 1), ('wan', 'a', 1), ('care', 'n', 1), ('Find', 'n', 1), ('time', 'n', 1), ('frighted', 'v', 1), ('peace', 'n', 1), ('pant', 'n', 1), ('breathe', 'n', 1), ('short-winded', 'a', 1), ('accents', 'n', 1), ('new', 'a', 1), ('broils', 'n', 1), ('commenced', 'v', 1), ('strands', 'n', 1), ('afar', 'a', 1), ('remote', 'v', 1), ('thirsty', 'a', 1), ('entrance', 'n', 1), ('soil', 'n', 1), ('Shall', 'n', 1), ('daub', 'v', 1), ('lips', 'n', 1), ('childrens', 'n', 1), ('blood', 'v', 1), ('shall', 'n', 1), ('trenching', 'v', 1), ('war', 'n', 1), ('channel', 'n', 1), ('fields', 'n', 1), ('bruise', 'v', 1), ('flowerets', 'n', 1), ('armed', 'v', 1), ('hoofs', 'a', 1), ('hostile', 'n', 1), ('paces', 'n', 1), ('oppose

In [ ]:
#Adequando o data frame

juntando = [tupla for array in classified_plays_with_index for tupla in array]
tabela = sc.createDataFrame(pd.DataFrame(juntando, columns=['PALAVRA', 'TIPO', 'HISTORIA']))


In [ ]:
#Mostrando a Tabela
tabela.show()

+------------+----+--------+
|     PALAVRA|TIPO|HISTORIA|
+------------+----+--------+
|      London|   n|       1|
|      palace|   n|       1|
|       Enter|   n|       1|
|        KING|   n|       1|
|       HENRY|   n|       1|
|        LORD|   n|       1|
|        JOHN|   n|       1|
|   LANCASTER|   n|       1|
|        EARL|   n|       1|
|WESTMORELAND|   n|       1|
|         SIR|   n|       1|
|      WALTER|   n|       1|
|       BLUNT|   n|       1|
|      others|   n|       1|
|      shaken|   v|       1|
|         wan|   a|       1|
|        care|   n|       1|
|        Find|   n|       1|
|        time|   n|       1|
|    frighted|   v|       1|
+------------+----+--------+
only showing top 20 rows



In [ ]:
#Testando query
tabela.createOrReplaceTempView("tabela_temp")
resultado = sc.sql("SELECT * FROM tabela_temp WHERE historia = 1")
resultado.show()

+------------+----+--------+
|     PALAVRA|TIPO|HISTORIA|
+------------+----+--------+
|      London|   n|       1|
|      palace|   n|       1|
|       Enter|   n|       1|
|        KING|   n|       1|
|       HENRY|   n|       1|
|        LORD|   n|       1|
|        JOHN|   n|       1|
|   LANCASTER|   n|       1|
|        EARL|   n|       1|
|WESTMORELAND|   n|       1|
|         SIR|   n|       1|
|      WALTER|   n|       1|
|       BLUNT|   n|       1|
|      others|   n|       1|
|      shaken|   v|       1|
|         wan|   a|       1|
|        care|   n|       1|
|        Find|   n|       1|
|        time|   n|       1|
|    frighted|   v|       1|
+------------+----+--------+
only showing top 20 rows



In [ ]:
#Exercício
#PERCENTUAL DA FREQUENCIA DA PALAVRA (PESO) POR PALAVRA E PELA HISTÓRIA
total_palavras = tabela.count()
resultado = sc.sql("SELECT palavra, historia,  COUNT(*) AS frequencia,(COUNT(*) * 100.0 / {}) AS percentual FROM tabela_temp GROUP BY palavra, historia Order By historia, frequencia Desc".format(total_palavras))
resultado.show()


+-------+--------+----------+----------+
|palavra|historia|frequencia|percentual|
+-------+--------+----------+----------+
|   thou|       1|       231|0.05553913|
|  shall|       1|       116|0.02788978|
|   thee|       1|       103|0.02476420|
|    thy|       1|       102|0.02452377|
|    Ill|       1|        87|0.02091733|
|   lord|       1|        79|0.01899390|
|   good|       1|        70|0.01683004|
|     us|       1|        66|0.01586832|
|   king|       1|        63|0.01514703|
|    man|       1|        60|0.01442575|
|   like|       1|        59|0.01418532|
|  would|       1|        57|0.01370446|
|   well|       1|        56|0.01346403|
|   upon|       1|        55|0.01322360|
|    art|       1|        55|0.01322360|
|   come|       1|        54|0.01298317|
|  Enter|       1|        50|0.01202146|
|   hath|       1|        48|0.01154060|
|   Lord|       1|        45|0.01081931|
|    let|       1|        45|0.01081931|
+-------+--------+----------+----------+
only showing top

In [ ]:
#Mostrando pelo tipo de palavra, onde quase 60% é de substantivo.
resultado = sc.sql("SELECT TIPO,  COUNT(*) AS frequencia,(COUNT(*) * 100.0 / {}) AS percentual FROM tabela_temp GROUP BY TIPO Order By frequencia Desc".format(total_palavras))
resultado.show()

+----+----------+-----------+
|TIPO|frequencia| percentual|
+----+----------+-----------+
|   n|    247262|59.44898455|
|   v|     77739|18.69071920|
|   a|     68260|16.41169159|
|   r|     22662| 5.44860467|
+----+----------+-----------+



In [ ]:
resultado = sc.sql("SELECT PALAVRA,  COUNT(*) QUANTIDADE FROM tabela_temp WHERE UPPER(TIPO) LIKE 'N' Group By PALAVRA ORDER BY COUNT(*) DESC" )
resultado.show()

+-------+----------+
|PALAVRA|QUANTIDADE|
+-------+----------+
|  shall|      3070|
|   thou|      2939|
|  would|      1995|
|  Enter|      1922|
|    man|      1747|
|    Ill|      1744|
|   thee|      1714|
|   lord|      1676|
|     us|      1630|
|    one|      1561|
|   like|      1462|
|    may|      1401|
|   hath|      1359|
|   must|      1336|
|   upon|      1290|
| Exeunt|      1019|
|    sir|      1012|
|   time|       932|
|   love|       914|
|   Exit|       911|
+-------+----------+
only showing top 20 rows

